In [ ]:
# -*- coding: utf-8 -*-
"""
File: hhl_example.ipynb
Author: Eddie Kelly
Date: 2024

This file is part of the Quantum algorithm for linear systems of equations for the multi-dimensional Black-Scholes equations project which was completed as part 
of the thesis https://mural.maynoothuniversity.ie/id/eprint/19288/.

License: MIT License
"""

import os
import sys

sys.path.append('..')

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import numpy as np
from qiskit import QuantumCircuit 
from qiskit_aer import Aer, AerSimulator
from qiskit.visualization import plot_histogram

from hhl import hhl, HHL_Solver
from ancillacircuit import eigenvalueextract_range

In [4]:
#-----------------Parameters for HHL circuit size - vector register-----------------#

#Components of vector to be solved for
a = 6
b = 8
vector = np.array([a,b])
#Matrix definiition

matrix = np.array([[1, 4], [4, 2]]) #Hermitian matrix to be inverted

#------------Parameters for HHL circuit selection - QPE-----------------#

t = 2*np.pi*np.linalg.cond(matrix)

M_prime = int(2**(np.ceil(np.log2(2*np.linalg.cond(matrix)))))

# If you want to use a specific M value, overwrite between these two comments
M = 32
#~~~~~~~           

M = max(M_prime,M) # Just taking the maximum of the two values.

qpe_reg_num_qubits = int(np.log2(M))

vector_reg_num_qubits = int(np.log2(len(matrix)))

print("---")
print("The time parameter (as the lower bound)            :",np.round(t,2))
print("Acceptable range for time parameter                :(%f, %f)"%(np.round(t,1),np.round(np.pi*M,1)))

# If you want to use a specific t value, overwrite below
# t = 18
#~~~~~~~                                        

print("The time parameter used                            :",t)
print("The lower bound for number of Fourier basis states :",M_prime)
print("Number of Fourier basis states used                :",M)
print("Number of qubits for QPE                           :",qpe_reg_num_qubits)

if (t > np.pi*M):
    raise ValueError("Time parameter is too large for the number of qubits.")

#-----------------Parameters for HHL circuit  - qubit lists-----------------#

ancilla_qubit = [0]

qpe_qubits = [i for i in range(1,qpe_reg_num_qubits + 1)]

vector_qubits = [i + (qpe_reg_num_qubits + 1)  for i in range(0,vector_reg_num_qubits)]

classical_vector_bits = [i for i in range(1,vector_reg_num_qubits+1)]


print("---")
print("Total number of qubits   :",vector_reg_num_qubits+qpe_reg_num_qubits+1)
print("These are ancilla qubit  :",ancilla_qubit)
print("These are qpe qubits     :",qpe_qubits)
print("These are vector qubits  :",vector_qubits)

#-----------------Parameters for HHL circuit - ancilla normalisation and eigenvalue tolerances-----------------#

eps = 0.01
C = (2*np.pi)/t - eps

abs_eigval_lower = 0       # Not actually imposing any lower bound.

abs_eigval_upper = np.inf  # Not actually imposing any upper bound.

print("Normalisation of ancilla :",np.round(C,3))

#-----------------Parameters for HHL circuit      - initialize circuit-----------------#


# circ = QuantumCircuit(vector_reg_num_qubits+qpe_reg_num_qubits+1,vector_reg_num_qubits+1)
# circ.initialize([a,b],vector_qubits,normalize=True)

#-----------------Parameters for HHL circuit size - actual HHL circuit-----------------#

---
The time parameter (as the lower bound)            : 13.73
Acceptable range for time parameter                :(13.700000, 100.500000)
The time parameter used                            : 13.730279808787762
The lower bound for number of Fourier basis states : 8
Number of Fourier basis states used                : 32
Number of qubits for QPE                           : 5
---
Total number of qubits   : 7
These are ancilla qubit  : [0]
These are qpe qubits     : [1, 2, 3, 4, 5]
These are vector qubits  : [6]
Normalisation of ancilla : 0.448


In [5]:
problem1 = HHL_Solver(matrix, M, t, C, abs_eigval_lower, abs_eigval_upper)
pic1 = problem1.circuitpic(vector, style_ = 'text', meas_sty='overlapmeas',second_state=np.array([4,3]))

                                                                      »
q_0: ─────────────────────────────────────────────────────────────────»
              ┌───┐                                                   »
q_1: ─────────┤ H ├───────────────────────────────────────────────────»
              ├───┤                                                   »
q_2: ─────────┤ H ├───────────────────────────────────────────────────»
              ├───┤                                                   »
q_3: ─────────┤ H ├───────────────────────────────────────────────────»
              ├───┤                                                   »
q_4: ─────────┤ H ├────────────────────────────────────────■──────────»
              ├───┤                                        │          »
q_5: ─────────┤ H ├───────────────────■────────────────────┼──────────»
     ┌────────┴───┴────────┐┌─────────┴─────────┐┌─────────┴─────────┐»
q_6: ┤ Initialize(0.6,0.8) ├┤ exp(+iH(t_0/M)*1) ├┤ exp(+iH(t_0/M

In [6]:
result2 = problem1.solve_by_qasm(vector, shots_ = 100000)

      -@-@-@-@-@-@-@-@-@-@-@-@-@-@-@-@-@-@-@-@- 
      |   ---------------------------------   | 
      |   * * * Classical Information * * *   | 
      |   ---------------------------------   | 
      -@-@-@-@-@-@-@-@-@-@-@-@-@-@-@-@-@-@-@-@-

The matrix to be inverted                                :

 [[1 4]
 [4 2]] 

The (un-normalized) input state |b> &  norm |||b>||      : [6 8], ||.|| = 10.000000

The input state normalized  |~b> =|b>/|||b>||            : [0.6 0.8]

     -----------------------------------------------------------------
     Defining classical actual solution state as  |x_acc> = A^-1 |b>  
     Defining (un-normalized)  solution state as  |x'> = A^-1 |~b>    
     Defining normalized solution state as        |~x> = |x'>/|||x'>||
     -----------------------------------------------------------------

The actual solution state  |x_acc> & norm |||x_acc>||    : [1.42857143 1.14285714], ||.|| = 1.829464

The (un-normalized) solution state  |x'> & norm |||x'>|| : [0.14